In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

In [2]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 35

Step 1: Read in data<br>
using TF Learn's built in function to load MNIST data to the folder data/mnist

In [3]:
mnist = input_data.read_data_sets('data/mnist', one_hot=True) 

Extracting data/mnist/train-images-idx3-ubyte.gz
Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz


Step 2: create placeholders for features and labels<br>
each image in the MNIST data is of shape 28*28 = 784<br>
therefore, each image is represented with a 1x784 tensor<br>
there are 10 classes for each image, corresponding to digits 0 - 9.<br>
Features are of the type float, and labels are of the type int<br>

In [4]:
X = tf.placeholder(tf.float32, shape=(None, 784), name = "X")
Y = tf.placeholder(tf.float32, shape=(None, 10), name = "Y")

Step 3: create weights and bias<br>
weights and biases are initialized to 0<br>
shape of w depends on the dimension of X and Y so that Y = X * w + b<br>
shape of b depends on Y<br>

In [5]:
#b = tf.Variable(tf.zeros(shape=(10)))
#w = tf.Variable(tf.zeros(shape=(784,10)))

b = tf.Variable(tf.random_normal(shape=[10], stddev=0.05), name = "b")
w = tf.Variable(tf.random_normal(shape=[784,10], stddev=0.05), name = "w")

Step 4: build model<br>
the model that returns the logits.<br>
this logits will be later passed through softmax layer<br>
to get the probability distribution of possible label of the image<br>
DO NOT DO SOFTMAX HERE<br>

In [6]:
logits = tf.matmul(X, w) + b

Step 5: define loss function<br>
use cross entropy loss of the real labels with the softmax of logits<br>
use the method:<br>
tf.nn.softmax_cross_entropy_with_logits(logits, Y)<br>
then use tf.reduce_mean to get the mean loss of the batch<br>

In [7]:
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name = "loss")
mean = tf.reduce_mean(loss, name = "loss_mean")

Step 6: define training op<br>
using gradient descent to minimize loss

In [8]:
opt = (tf.train.GradientDescentOptimizer(learning_rate)).minimize(mean, name = "opt")

with tf.Session() as sess:
	start_time = time.time()
	sess.run(tf.global_variables_initializer())	
	n_batches = int(mnist.train.num_examples/batch_size)
	for i in range(n_epochs): # train the model n_epochs times
		total_loss = 0

		for _ in range(n_batches):
			X_batch, Y_batch = mnist.train.next_batch(batch_size)
			
			_,loss_batch = sess.run([opt, mean], feed_dict={X:X_batch, Y:Y_batch})
			total_loss += loss_batch
		print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

	print('Total time: {0} seconds'.format(time.time() - start_time))

	print('Optimization Finished!') # should be around 0.35 after 25 epochs

	# test the model
	preds = tf.nn.softmax(logits)
	correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
	accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # need numpy.count_nonzero(boolarr) :(
	
	n_batches = int(mnist.test.num_examples/batch_size)
	total_correct_preds = 0
	
	for i in range(n_batches):
		X_batch, Y_batch = mnist.test.next_batch(batch_size)
		accuracy_batch = sess.run([accuracy], feed_dict={X: X_batch, Y:Y_batch}) 
		total_correct_preds += accuracy_batch[0]
	
	print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

Average loss epoch 0: 1.3338024325859852
Average loss epoch 1: 0.7472463590559704
Average loss epoch 2: 0.6097073436200202
Average loss epoch 3: 0.5434367309361349
Average loss epoch 4: 0.5037760941576569
Average loss epoch 5: 0.47620376618036303
Average loss epoch 6: 0.45642632968498
Average loss epoch 7: 0.43929484000294916
Average loss epoch 8: 0.42752696907186843
Average loss epoch 9: 0.4163512593104845
Average loss epoch 10: 0.40826535874452347
Average loss epoch 11: 0.3992256983454689
Average loss epoch 12: 0.3937506363455788
Average loss epoch 13: 0.38713979443192204
Average loss epoch 14: 0.380808454015594
Average loss epoch 15: 0.3780488070352372
Average loss epoch 16: 0.37255765185072703
Average loss epoch 17: 0.36824117531448536
Average loss epoch 18: 0.3653256537822577
Average loss epoch 19: 0.36228231978027414
Average loss epoch 20: 0.3588094764795059
Average loss epoch 21: 0.3561849155417689
Average loss epoch 22: 0.3527647639626945
Average loss epoch 23: 0.35101852810883